In [9]:
ofns_desc_values_path = './datasets/column_values_for_clearing/ofns_desc_values.txt'
prem_typ_values_path = './datasets/column_values_for_clearing/prem_typ_values.txt'
with open(ofns_desc_values_path) as f:
    columns = f.read().split('\n')
print(len(columns))

73


In [10]:
columns = {i: column for i,column in enumerate(columns)}
columns1 = columns.copy()

In [3]:
#Universal Sentence Encoder
import tensorflow as tf
from tensorflow_hub import load
import tensorflow_text
import numpy as np

In [11]:
use_model = load('https://tfhub.dev/google/universal-sentence-encoder-multilingual/3')
# Define a function to get the embeddings of the product names
def get_embeddings(text_dict):#texts = {ID:text} 
    texts = list(text_dict.values())
    ids = list(text_dict.keys())
    # Create a tf.data.Dataset to load the texts in a memory-efficient way
    dataset = tf.data.Dataset.from_tensor_slices(texts)
    # Batch the dataset to reduce the memory required for processing the embeddings
    dataset = dataset.batch(2048)
    # Get the embeddings using the Universal Sentence Encoder
    embeddings = []
    for batch in texts:
        batch_embeddings = use_model(batch)
        # Convert the embeddings to float16 to reduce their size in memory
        batch_embeddings = tf.cast(batch_embeddings, tf.float16)
        embeddings.append(batch_embeddings)
    embeddings = np.concatenate(embeddings)
    embeddings_dict = {}
    for i, embedding in enumerate(embeddings):
        embeddings_dict[ids[i]] = embedding
        
    return embeddings_dict

In [12]:
# Get the embeddings of the product names in the databases
database1_embeddings = get_embeddings(columns)
database2_embeddings = get_embeddings(columns1)

# Compute the cosine similarity matrix between the product name embeddings
database1_ids, database1_embeddings = zip(*database1_embeddings.items())
database2_ids, database2_embeddings = zip(*database2_embeddings.items())

cosine_similarities = np.matmul(database1_embeddings, np.transpose(database2_embeddings))

# Find the most similar product names
max_similarities = np.max(cosine_similarities, axis=1)
max_indices = np.argmax(cosine_similarities, axis=1)

In [17]:
threshold = 0.6#strict >= 0.84, confident > 0.75, almost all > 0.52

def getTopMatches(top):
    global WantList
    WantList = True

    top_indices = np.argsort(cosine_similarities, axis=1)[:, ::-1][:, :top]
    top_similarities = np.sort(cosine_similarities, axis=1)[:, ::-1][:, :top]
    matched_products = {}
    
    for i, id1 in enumerate(database1_ids):
        id2_list = [database2_ids[j] for j in top_indices[i] if top_similarities[i][list(top_indices[i]).index(j)] >= threshold]
        matched_products[id1] = id2_list

    return matched_products

In [18]:
top_matches = getTopMatches(top=5)
for key in top_matches:
    print(columns[key], [columns[k] for k in top_matches[key] if k != key])

KIDNAPPING AND RELATED OFFENSES ['KIDNAPPING & RELATED OFFENSES', 'PROSTITUTION & RELATED OFFENSES', 'OFFENSES RELATED TO CHILDREN', 'CRIMINAL MISCHIEF & RELATED OF']
UNDER THE INFLUENCE OF DRUGS ['DANGEROUS DRUGS', 'LOITERING FOR DRUG PURPOSES']
OFFENSES AGAINST MARRIAGE UNCL ['OFFENSES AGAINST THE PERSON', 'OFFENSES AGAINST PUBLIC SAFETY']
ABORTION []
FORTUNE TELLING ['ANTICIPATORY OFFENSES', 'DISORDERLY CONDUCT', 'JOSTLING', 'LOITERING']
OTHER STATE LAWS (NON PENAL LAW) ['OTHER STATE LAWS (NON PENAL LA', 'OTHER STATE LAWS', 'MISCELLANEOUS PENAL LAW']
LOITERING/DEVIATE SEX ['SEX CRIMES', 'FELONY SEX CRIMES']
OTHER TRAFFIC INFRACTION ['OTHER OFFENSES RELATED TO THEF']
ADMINISTRATIVE CODES ['ADMINISTRATIVE CODE']
LOITERING FOR DRUG PURPOSES ['DANGEROUS DRUGS', 'UNDER THE INFLUENCE OF DRUGS']
LOITERING ['JOSTLING', 'FORGERY', 'FRAUDS', 'FORTUNE TELLING']
FELONY SEX CRIMES ['SEX CRIMES', 'LOITERING/DEVIATE SEX', 'FELONY ASSAULT']
DISRUPTION OF A RELIGIOUS SERV []
INTOXICATED/IMPAIRED DRI

In [ ]:
# ofns_desc
# changing to -> 
'PETIT LARCENY OF MOTOR VEHICLE', 'LARENCY OF MOTOR VEHICLE'
'GRAND LARCENY OF MOTOR VEHICLE', 'LARENCY OF MOTOR VEHICLE'
'UNAUTHORIZED USE OF A VEHICLE', 'LARENCY OF MOTOR VEHICLE'
'HARRASSMENT 2', 'HARRASSMENT'
'ASSAULT 3 & RELATED OFFENSES', 'ASSAULT'
'FELONY ASSAULT', 'ASSAULT'
'DANGEROUS DRUGS', 'DRUGS'
'CANNABIS RELATED OFFENSES', 'DRUGS'
'UNDER THE INFLUENCE OF DRUGS', 'DRUGS'
'OFF. AGNST PUB ORD SENSBLTY &', 'OFFENCE AGAINST PUBLIC ORDER & SENSIBILITY'
'DISRUPTION OF A RELIGIOUS SERV', 'OFFENCE AGAINST PUBLIC ORDER & SENSIBILITY'
'DISORDERLY CONDUCT', 'OFFENCE AGAINST PUBLIC ORDER & SENSIBILITY'
'BURGLAR\'S TOOLS', 'BURGLARY'
'OFFENSES AGAINST PUBLIC ADMINI', 'OFFENSES AGAINST PUBLIC SAFETY'
'OTHER TRAFFIC INFRACTION', 'VEHICLE AND TRAFFIC LAWS'
'INTOXICATED & IMPAIRED DRIVING','INTOXICATED/IMPAIRED DRIVING'
'FRAUDULENT ACCOSTING','FRAUDS'
'OFFENSES INVOLVING FRAUD', 'FRAUDS'
'FRAUDULENT ACCOSTING', 'FRAUDS'
'OTHER OFFENSES RELATED TO THEF', 'THEFT'
'THEFT-FRAUD', 'THEFT'
'THEFT OF SERVICES', 'THEFT'
'NYS LAWS-UNCLASSIFIED FELONY', 'FELONY'
'MURDER & NON-NEGL. MANSLAUGHTER', 'FELONY'
'RAPE', 'FELONY'
'KIDNAPPING', 'FELONY'
'ARSON', 'FELONY'
'LOITERING/DEVIATE SEX', 'SEX CRIMES'
'FELONY SEX CRIMES', 'SEX CRIMES'
'PROSTITUTION & RELATED OFFENSES', 'SEX CRIMES'
'KIDNAPPING AND RELATED OFFENSES', 'OFFENSES AGAINST THE PERSON'
'KIDNAPPING & RELATED OFFENSES', 'OFFENSES AGAINST THE PERSON'
'OFFENSES RELATED TO CHILDREN', 'OFFENSES AGAINST THE PERSON'
'CHILD ABANDONMENT/NON SUPPORT', 'OFFENSES AGAINST THE PERSON'
'OFFENSES AGAINST MARRIAGE UNCL', 'OFFENSES AGAINST THE PERSON'
'KIDNAPPING', 'OFFENSES AGAINST THE PERSON'
'ANTICIPATORY OFFENSES','OFFENSES AGAINST THE PERSON'
'CARDS','GAMBLING'
'LOITERING/GAMBLING', 'GAMBLING'
'ADMINISTRATIVE CODE', 'ADMINISTRATIVE OFFENSES'
'ADMINISTRATIVE CODES', 'ADMINISTRATIVE OFFENSES'
'UNLAWFUL POSS. WEAP. ON SCHOOL', 'DANGEROUS WEAPONS'


'AGRICULTURE & MRKTS LAW-UNCLASSIFIED', 'UNKNOWN'
'NYS LAWS-UNCLASSIFIED VIOLATION', 'UNKNOWN'
'NYS LAWS-UNCLASSIFIED FELONY', 'UNKNOWN'
'OTHER STATE LAWS (NON PENAL LA', 'UNKNOWN'
'OTHER STATE LAWS (NON PENAL LAW)', 'UNKNOWN'
'OTHER STATE LAWS', 'UNKNOWN'
'MISCELLANEOUS PENAL LAW', 'UNKNOWN'
'JOSTLING', 'UNKNOWN'
'ENDAN WELFARE INCOMP', 'UNKNOWN'
'LOITERING/GAMBLING (CARDS, DIC', 'UNKNOWN'
'ESCAPE 3', 'UNKNOWN'
'HOMICIDE-NEGLIGENT,UNCLASSIFIE', 'UNKNOWN'
'HOMICIDE-NEGLIGENT-VEHICLE', 'UNKNOWN'
'NEW YORK CITY HEALTH CODE', 'UNKNOWN'
'LOITERING', 'UNKNOWN'
'LOITERING FOR DRUG PURPOSES', 'UNKNOWN'
'FORTUNE TELLING', 'UNKNOWN'
'ABORTION', 'UNKNOWN'

In [ ]:
# prem_typ_desc
# changing to -> 
'BEAUTY & NAIL SALON','BEAUTY/NAIL SALON'
'CHECK CASHING BUSINESS','CHECK CASH'
'CLOTHING/BOUTIQUE', 'CLOTHING BOUTIQUE'
'COMMERCIAL BLDG', 'COMMERCIAL BUILDING'
'DEPARTMENT STORE', 'DEPT STORE'
'DOCTOR/DENTIST OFFICE','DOCTOR/DENTIST'
'FOOD SUPERMARKET','SUPERMARKET'
'JEWELRY STORE','JEWELRY'
'NONE',None
'(null)',None
'SHOE STORE','SHOE'
'SOCIAL CLUB/POLICY LOCATI','SOCIAL CLUB/POLICY'